In [1]:
import os

os.environ["JAX_PLATFORMS"] = "cpu" 


from datasets import Dataset

from gensbi_examples.tasks import get_task

from jax import numpy as jnp

import json
from huggingface_hub import upload_file

import numpy as np

# Your dictionary with metadata


/lhome/ific/a/aamerio/miniforge3/envs/gensbi/lib/python3.12/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at grain/proto/execution_summary.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(


In [2]:
import torch

In [3]:
tasks = ["two_moons", "slcp", "gaussian_linear", "gaussian_mixture"]
repo_name = "aurelio-amerio/SBI-benchmarks"

In [3]:
metadata = {}

for task_name in tasks:
    task = get_task(task_name)
    dim_data = task.data["dim_data"].item()
    dim_theta = task.data["dim_theta"].item()

    metadata[task_name] = {"dim_data": dim_data, "dim_theta": dim_theta}

file_path = "metadata.json"
with open(file_path, 'w') as f:
    json.dump(metadata, f, indent=4)

./task_data/data_two_moons.npz already exists, skipping download.
./task_data/data_slcp.npz already exists, skipping download.
./task_data/data_gaussian_linear_uniform.npz already exists, skipping download.
./task_data/data_gaussian_linear.npz already exists, skipping download.
./task_data/data_gaussian_mixture.npz already exists, skipping download.


In [4]:
upload_file(
    path_or_fileobj=file_path,
    path_in_repo="metadata.json",  # The name of the file in the repo
    repo_id=repo_name,
    repo_type="dataset",
)

CommitInfo(commit_url='https://huggingface.co/datasets/aurelio-amerio/SBI-benchmarks/commit/907cb05a9fbc842761e77c2a037cc9bc8efd0b52', commit_message='Upload metadata.json with huggingface_hub', commit_description='', oid='907cb05a9fbc842761e77c2a037cc9bc8efd0b52', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/aurelio-amerio/SBI-benchmarks', endpoint='https://huggingface.co', repo_type='dataset', repo_id='aurelio-amerio/SBI-benchmarks'), pr_revision=None, pr_num=None)

In [ ]:
# upload dataset function

def upload_dataset(task_name: str, repo_name: str):
    task = get_task(task_name)
    data_dict = dict(task.data)

    max_samples = int(1e6)
    dtype = jnp.float32

    xs = data_dict["xs"][: max_samples]
    xs = xs.astype(dtype)
    thetas = data_dict["thetas"][: max_samples]
    thetas = thetas.astype(dtype)

    xs_val = data_dict["xs"][max_samples :]
    xs_val = xs_val.astype(dtype)
    thetas_val = data_dict["thetas"][max_samples :]
    thetas_val = thetas_val.astype(dtype)

    observations = data_dict["observations"]
    observations = observations.astype(dtype)

    reference_samples = data_dict["reference_samples"]
    reference_samples = reference_samples.astype(dtype)

    true_parameters = data_dict["true_parameters"]
    true_parameters = true_parameters.astype(dtype)

    # dim_data = data_dict["dim_data"]
    # dim_theta = data_dict["dim_theta"]
    # dim_joint = dim_data + dim_theta
    # num_observations = data_dict["num_observations"]

    dataset_train = Dataset.from_dict({"xs": xs, "thetas": thetas})
    dataset_val = Dataset.from_dict({"xs": xs_val, "thetas": thetas_val})
    dataset_reference_posterior = Dataset.from_dict(
        {"reference_samples": reference_samples, "observations": observations, "true_parameters": true_parameters}
    )

    dataset_train.push_to_hub(repo_name, config_name=task_name, split="train", private=False)
    dataset_val.push_to_hub(repo_name, config_name=task_name, split="validation", private=False)
    dataset_reference_posterior.push_to_hub(repo_name, config_name=f"{task_name}_posterior", split="reference_posterior", private=False)

    return

In [3]:
dir_GW = "/lhome/ific/a/aamerio/data/GW"

In [4]:
thetas = torch.load(f"{dir_GW}/thetas_0.pt")
theta1 = torch.load(f"{dir_GW}/thetas_1.pt")
theta2 = torch.load(f"{dir_GW}/thetas_2.pt")
theta3 = torch.load(f"{dir_GW}/thetas_3.pt")
theta4 = torch.load(f"{dir_GW}/thetas_4.pt")
theta5 = torch.load(f"{dir_GW}/thetas_5.pt")
theta6 = torch.load(f"{dir_GW}/thetas_6.pt")
theta7 = torch.load(f"{dir_GW}/thetas_7.pt")
theta8 = torch.load(f"{dir_GW}/thetas_8.pt")
theta9 = torch.load(f"{dir_GW}/thetas_9.pt")

xs_raw = torch.load(f"{dir_GW}/xs_0.pt")
xs_raw1 = torch.load(f"{dir_GW}/xs_1.pt")
xs_raw2 = torch.load(f"{dir_GW}/xs_2.pt")
xs_raw3 = torch.load(f"{dir_GW}/xs_3.pt")
xs_raw4 = torch.load(f"{dir_GW}/xs_4.pt")
xs_raw5 = torch.load(f"{dir_GW}/xs_5.pt")
xs_raw6 = torch.load(f"{dir_GW}/xs_6.pt")
xs_raw7 = torch.load(f"{dir_GW}/xs_7.pt")
xs_raw8 = torch.load(f"{dir_GW}/xs_8.pt")
xs_raw9 = torch.load(f"{dir_GW}/xs_9.pt")

In [5]:
thetas = torch.cat([thetas, theta1, theta2, theta3, theta4, theta5, theta6, theta7, theta8, theta9], dim=0)
xs_raw = torch.cat([xs_raw, xs_raw1, xs_raw2, xs_raw3, xs_raw4, xs_raw5, xs_raw6, xs_raw7, xs_raw8, xs_raw9], dim=0)

In [6]:
thetas = thetas.numpy()
xs_raw = xs_raw.numpy()

In [7]:
xs_train = xs_raw[:-512]
xs_val = xs_raw[-512:]
thetas_train = thetas[:-512]
thetas_val = thetas[-512:]

In [9]:
dataset_train = Dataset.from_dict({"xs": xs_train, "thetas": thetas_train})
dataset_val = Dataset.from_dict({"xs": xs_val, "thetas": thetas_val})

: 

In [ ]:
dataset_train.push_to_hub(repo_name, config_name="gravitational_waves", split="train", private=False)
dataset_val.push_to_hub(repo_name, config_name="gravitational_waves", split="validation", private=False)